In [1]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [7]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 37.5 MB/s eta 0:00:00


In [2]:
# Define file paths
PYTORCH_MODEL_PATH = "sentiment_model.pt"
ONNX_MODEL_PATH = "sentiment_model.onnx"
TRT_FP32_PATH = "sentiment_model_fp32.plan"
TRT_FP16_PATH = "sentiment_model_fp16.plan"

In [3]:
# 1. Save PyTorch model (.pt)
def save_pytorch_model(model):
    torch.save(model.state_dict(), PYTORCH_MODEL_PATH)
    print(f"PyTorch model saved at {PYTORCH_MODEL_PATH}")

In [4]:
# 2. Convert to ONNX (.onnx)
def convert_to_onnx(model, tokenizer):
    model.eval()
    # Example input to trace the model (dummy input)
    text = "The service was great!"
    inputs = tokenizer(text, return_tensors="pt")

    torch.onnx.export(
        model,
        (inputs['input_ids'], inputs['attention_mask']),  # model input tuple
        ONNX_MODEL_PATH,
        export_params=True,
        opset_version=14,
        input_names=['input_ids', 'attention_mask'],
        output_names=['output'],
        dynamic_axes={'input_ids': {0: 'batch_size'}, 'attention_mask': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
    )
    print(f"ONNX model saved at {ONNX_MODEL_PATH}")

In [9]:
# 3. Convert ONNX to TensorRT using trtexec
def convert_to_tensorrt(onnx_model_path, trt_fp32_path, trt_fp16_path):
    # Convert ONNX to TensorRT FP32
    fp32_cmd = f"trtexec --onnx={onnx_model_path} --saveEngine={trt_fp32_path}"
    os.system(fp32_cmd)
    print(f"TensorRT FP32 model saved at {trt_fp32_path}")

    # Convert ONNX to TensorRT FP16
    fp16_cmd = f"trtexec --onnx={onnx_model_path} --saveEngine={trt_fp16_path} --fp16" # onnx_model_path is now defined in the scope of the function.
    os.system(fp16_cmd)
    print(f"TensorRT FP16 model saved at {trt_fp16_path}")

In [10]:

# Main process
def main():
    # Load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
    model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

    # Step 1: Save PyTorch Model
    save_pytorch_model(model)

    # Step 2: Convert to ONNX
    convert_to_onnx(model, tokenizer)

    # Step 3: Convert to TensorRT (FP32 and FP16)
    convert_to_tensorrt(ONNX_MODEL_PATH, TRT_FP32_PATH, TRT_FP16_PATH)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


PyTorch model saved at sentiment_model.pt
ONNX model saved at sentiment_model.onnx
TensorRT FP32 model saved at sentiment_model_fp32.plan
TensorRT FP16 model saved at sentiment_model_fp16.plan
